In [ ]:
import os
import re
import glob
from datetime import datetime
import sys
from collections import defaultdict

import pandas as pd
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
sys.path.append("..") # Adds higher directory to python modules path for importing from src dir

from src.algorithms import SimpleLSTM, MLPClassifier, SVC
from src.datasets import NyseStocksDataset
from src.evaluation import Evaluator, Plotter
from src.utils import print_metrics_as_md, pandas_df_to_markdown_table
# from src.pipeline import run_pipeline, prepare_data, evaluate_results
# from src.experiments import get_metrics

%matplotlib inline
%load_ext autotime
%load_ext autoreload
%autoreload 2

In [ ]:
plotter = Plotter('../reports')

# Experiments

## 1. Feature selection
- MLP & SimpleLSTM
- All-2C, All-3C, OC-3C, OHLC-3C, Open-3C

Winner: __OC-3C__

|  -   |     All-2C    MLP   |     All-2C    SLSTM_shuffle   |     All-3C    MLP   |     All-3C    SLSTM_shuffle   |     OC-3C    MLP   |     OC-3C    SLSTM_shuffle   |     OHLC-3C    MLP   |     OHLC-3C    SLSTM_shuffle   |     Open-3C    MLP   |     Open-3C    SLSTM_shuffle   |
|:-----|--------------------:|------------------------------:|--------------------:|------------------------------:|-------------------:|-----------------------------:|---------------------:|-------------------------------:|---------------------:|-------------------------------:|
| prec |           0.473675  |                     0.451195  |           0.330751  |                     0.298249  |          0.368896  |                    0.340891  |            0.320769  |                     0.33705    |           0.369802   |                      0.342179  |
| rec  |           0.4825    |                     0.451833  |           0.310833  |                     0.309333  |          0.344833  |                    0.340833  |            0.305     |                     0.336333   |           0.337      |                      0.3395    |
| f1   |           0.435161  |                     0.450036  |           0.308623  |                     0.297981  |          0.300402  |                    0.316042  |            0.287787  |                     0.314503   |           0.234739   |                      0.308558  |
| acc  |           0.4825    |                     0.451833  |           0.310833  |                     0.309333  |          0.344833  |                    0.340833  |            0.305     |                     0.336333   |           0.337      |                      0.3395    |
| mcc  |          -0.0429275 |                    -0.0969692 |          -0.0348393 |                    -0.0367314 |          0.0213947 |                    0.0120531 |           -0.0450657 |                     0.00477629 |           0.00990098 |                      0.0101475 |

[comment]: <> (Execution Code: print_metrics_as_md "../reports/custom/feature-selection.csv")

## 2. Number of Classes
- MLP & SimpleLSTM & SimpleLSTMShuffle & SVC
- All-2C, All-3C, OC-2C, OC-3C

Winner: __OC-3C__


| -  |   datasets    predictors     |     All-2C    MLP   |     All-2C    SLSTM   |     All-2C    SLSTM_shuffle   |     All-2C    SVC   |     All-3C    MLP   |     All-3C    SLSTM   |     All-3C    SLSTM_shuffle   |     All-3C    SVC   |     OC-2C    MLP   |     OC-2C    SLSTM   |     OC-2C    SLSTM_shuffle   |     OC-2C    SVC   |     OC-3C    MLP   |     OC-3C    SLSTM   |     OC-3C    SLSTM_shuffle   |     OC-3C    SVC   |
|---:|:-----------------------------|--------------------:|----------------------:|------------------------------:|--------------------:|--------------------:|----------------------:|------------------------------:|--------------------:|-------------------:|---------------------:|-----------------------------:|-------------------:|-------------------:|---------------------:|-----------------------------:|-------------------:|
|  0 | prec                         |             0.47367 |               0.25    |                       0.46111 |             0.46438 |             0.33075 |               0.11111 |                       0.30102 |             0.3193  |            0.50449 |              0.25    |                      0.50802 |            0.49017 |            0.3689  |              0.11111 |                      0.34566 |            0.34713 |
|  1 | rec                          |             0.4825  |               0.5     |                       0.466   |             0.46883 |             0.31083 |               0.33333 |                       0.31167 |             0.33267 |            0.502   |              0.5     |                      0.508   |            0.49017 |            0.34483 |              0.33333 |                      0.33467 |            0.34733 |
|  2 | f1                           |             0.43516 |               0.33333 |                       0.44866 |             0.45169 |             0.30862 |               0.16667 |                       0.29172 |             0.31268 |            0.42185 |              0.33333 |                      0.50775 |            0.49017 |            0.3004  |              0.16667 |                      0.28432 |            0.33982 |
|  3 | acc                          |             0.4825  |               0.5     |                       0.466   |             0.46883 |             0.31083 |               0.33333 |                       0.31167 |             0.33267 |            0.502   |              0.5     |                      0.508   |            0.49017 |            0.34483 |              0.33333 |                      0.33467 |            0.34733 |
|  4 | mcc                          |            -0.04293 |               0       |                      -0.07273 |            -0.06664 |            -0.03484 |               0       |                      -0.03433 |            -0.00104 |            0.00599 |              0       |                      0.01602 |           -0.01967 |            0.02139 |              0       |                      0.00225 |            0.02146 |


[comment]: <> (Execution Code: print_metrics_as_md "../reports/custom/classes-2vs3.csv")

## 3. Test on Many Samples

- MLP & SimpleLSTMShuffle
- All-3C, OC-3C
- Samples: 600k

Winner: __All-3C, MLP__

When training with more samples it's useful to provide more features.
Still wondering why LSTM isn't better than MLP - might be because LSTM didn't converge (see history plots). The val_acc didn't improve until 20th epoch and afterwards started oszillation intensively (with an increasing mean).

| -  |   datasets    predictors     |     All-3C    MLP   |     All-3C    SLSTM_shuffle   |     OC-3C    MLP   |     OC-3C    SLSTM_shuffle   |
|---:|:-----------------------------|--------------------:|------------------------------:|-------------------:|-----------------------------:|
|  0 | prec                         |             0.39888 |                       0.37705 |            0.36527 |                      0.38532 |
|  1 | rec                          |             0.40535 |                       0.35013 |            0.36832 |                      0.35853 |
|  2 | f1                           |             0.39455 |                       0.29944 |            0.33878 |                      0.30229 |
|  3 | acc                          |             0.40535 |                       0.35013 |            0.36832 |                      0.35853 |
|  4 | mcc                          |             0.10996 |                       0.03225 |            0.05683 |                      0.04865 |

[comment]: <> (Execution Code: print_metrics_as_md "../reports/custom/600k-samples.csv")

## 4. Why does not Shuffling Fail for LSTM?

- SimpleLSTM & SimpleLSTMShuffle
- OC-3C

1st Run: learning rate=0.1

| -  |   datasets    predictors     |     OC-3C    SLSTM   |     OC-3C    SLSTM_shuffle   |
|---:|:-----------------------------|---------------------:|-----------------------------:|
|  0 | prec                         |              0.11111 |                      0.2377  |
|  1 | rec                          |              0.33333 |                      0.34017 |
|  2 | f1                           |              0.16667 |                      0.23521 |
|  3 | acc                          |              0.33333 |                      0.34017 |
|  4 | mcc                          |              0       |                      0.01705 |

2nd Run: learning rate=0.01

| -  |   datasets    predictors     |     OC-3C    SLSTM   |     OC-3C    SLSTM_shuffle   |
|---:|:-----------------------------|---------------------:|-----------------------------:|
|  0 | prec                         |              0.11111 |                      0.30789 |
|  1 | rec                          |              0.33333 |                      0.32    |
|  2 | f1                           |              0.16667 |                      0.26147 |
|  3 | acc                          |              0.33333 |                      0.32    |
|  4 | mcc                          |              0       |                     -0.0231  |

Winner: __SLSTM_shuffle[lr=0.1]__

[comment]: <> (Execution Code: print_metrics_as_md "../reports/custom/shuffling.csv")
[comment]: <> (Execution Code: print_metrics_as_md "../reports/custom/shuffling.csv-1")

## 5. Good batch sizes for SLSTM

- SimpleLSTM [bs = 16 - 512]
- OC-3C, All-3C

The history of the approaches showed dann a very high batch size leeds to a saver convergence. But it's only converging for train and val data. Test set remains bad. I guess, the models with high batch size are more stable but also totally overfit.

|   -    |   datasets    predictors     |     OC-3C    SLSTM_bs16   |     OC-3C    SLSTM_bs32   |     OC-3C    SLSTM_bs64   |     OC-3C    SLSTM_bs128   |     OC-3C    SLSTM_bs256   |
|:-------|:-----------------------------|--------------------------:|--------------------------:|--------------------------:|---------------------------:|---------------------------:|
| 0      | prec                         |                   0.36755 |                   0.3343  |                   0.35211 |                    0.35629 |                    0.3551  |
| 1      | rec                          |                   0.34544 |                   0.34344 |                   0.358   |                    0.35622 |                    0.35667 |
| 2      | f1                           |                   0.24508 |                   0.24422 |                   0.32332 |                    0.3493  |                    0.34756 |
| 3      | acc                          |                   0.34544 |                   0.34344 |                   0.358   |                    0.35622 |                    0.35667 |
| 4      | mcc                          |                   0.028   |                   0.02431 |                   0.04044 |                    0.03506 |                    0.03575 |
| bs     | 0                            |                  16       |                  32       |                  64       |                  256       |                  512       |
| epochs | 0                            |                  50       |                 100       |                 150       |                  200       |                  250       |


Winner: __SLSTM_bs64__

[comment]: <> (Execution Code: custom_print_md)

In [ ]:
def custom_print_md():
    df = pd.read_csv("../reports/custom/lstm-batch-size-val-shuffled-lr001.csv", header=[0, 1])
    # df.loc['bs'] = int(re.search(r'[0-9]+$', x).group(0)
    df.loc['bs'] = [0, 256, 16, 512, 32, 64]
    df.loc['epochs'] = [0, 200, 50, 250, 100, 150]
    df.sort_values(axis=1, by='bs', inplace=True)
    print(pandas_df_to_markdown_table(df))
# custom_print_md()

## 6. Many Epochs

- SimpleLSTM [bs = 64 - 512] with 500 epochs
- 90k/9k from OC-3C

| -  |   datasets    predictors     |     OC-3C    SLSTM_bs64   |     OC-3C    SLSTM_bs256   |     OC-3C    SLSTM_bs512   |
|:---|:-----------------------------|--------------------------:|---------------------------:|---------------------------:|
| 0  | prec                         |                   0.23335 |                    0.34228 |                    0.34124 |
| 1  | rec                          |                   0.34089 |                    0.34444 |                    0.34256 |
| 2  | f1                           |                   0.22251 |                    0.34098 |                    0.3359  |
| 3  | acc                          |                   0.34089 |                    0.34444 |                    0.34256 |
| 4  | mcc                          |                   0.02126 |                    0.01678 |                    0.01407 |
| bs | 0                            |                  64       |                  256       |                  512       |

Winner: __SLSTM_bs64[epochs=500, lr=0.01]__

Outcome: Validation set is not good selected (models overfit)

[comment]: <> (Execution Code: custom_print_md)

In [ ]:
def custom_print_md():
    df = pd.read_csv("../reports/custom/lstm-many-epochs.csv", header=[0, 1])
    # df.loc['bs'] = int(re.search(r'[0-9]+$', x).group(0)
    df.loc['bs'] = [0, 256, 512, 64]
    df.sort_values(axis=1, by='bs', inplace=True)
    print(pandas_df_to_markdown_table(df))
# custom_print_md()

## 7. Good epsilon for `still` class

- SimpleLSTM_bs64, SimpleLSTM_bs512 with 200 epochs
- OC-3C[epsilon=0.01], OC-3C[epsilon=0.007], OC-3C[epsilon=0.004], OC-3C[epsilon=0.0005], OC-3C[epsilon=0.0005]

Outcome: Failed after first 0.0005 run (classes were not even distributed). Problem gets easier with growing size of `still` class.


| -  |   datasets    predictors     |     OC-3C-e01    SLSTM_bs64   |     OC-3C-e01    SLSTM_bs512   |     OC-3C-e007    SLSTM_bs64   |     OC-3C-e007    SLSTM_bs512   |     OC-3C-e004    SLSTM_bs64   |     OC-3C-e004    SLSTM_bs512   |     OC-3C-e001    SLSTM_bs64   |     OC-3C-e001    SLSTM_bs512   |     OC-3C-e0005    SLSTM_bs64   |     OC-3C-e0005    SLSTM_bs512   |     OC-3C-e0001    SLSTM_bs64   |
|---:|:-----------------------------|------------------------------:|-------------------------------:|-------------------------------:|--------------------------------:|-------------------------------:|--------------------------------:|-------------------------------:|--------------------------------:|--------------------------------:|---------------------------------:|--------------------------------:|
|  0 | prec                         |                       0.24276 |                        0.3983  |                        0.34473 |                         0.38659 |                        0.35464 |                         0.33107 |                        0.36028 |                         0.36562 |                         0.21973 |                          0.39352 |                         0.48808 |
|  1 | rec                          |                       0.346   |                        0.41417 |                        0.33983 |                         0.39567 |                        0.343   |                         0.33    |                        0.34217 |                         0.3695  |                         0.33317 |                          0.393   |                         0.48963 |
|  2 | f1                           |                       0.21848 |                        0.39045 |                        0.24143 |                         0.36906 |                        0.24607 |                         0.31759 |                        0.27598 |                         0.32588 |                         0.25538 |                          0.36731 |                         0.47724 |
|  3 | acc                          |                       0.346   |                        0.41417 |                        0.33983 |                         0.39567 |                        0.343   |                         0.33    |                        0.34217 |                         0.3695  |                         0.33317 |                          0.393   |                         0.48963 |
|  4 | mcc                          |                       0.03987 |                        0.12608 |                        0.01472 |                         0.09992 |                        0.0237  |                        -0.00522 |                        0.01598 |                         0.0629  |                        -0.00031 |                          0.09711 |                        -0.02097 |

Winner: __OC-3C[e=0.01]__

[comment]: <> (Execution Code: print_metrics_as_md "../reports/custom/lstm-epsilon.csv")

# Final

- SimpleLSTM, MLP
- OC-3C

| -  |   datasets    predictors     |     OC-3C    MLP   |     OC-3C    SLSTM   |
|---:|:-----------------------------|-------------------:|---------------------:|
|  0 | prec                         |            0.34613 |              0.3394  |
|  1 | rec                          |            0.34507 |              0.33757 |
|  2 | f1                           |            0.34058 |              0.32491 |
|  3 | acc                          |            0.34507 |              0.33757 |
|  4 | mcc                          |            0.01784 |              0.00658 |

Outcome: Might fail because 200 epochs are not enough on 300k training data.


In [ ]:
datasets = [
    NyseStocksDataset('OC-3C', file_path='../data/nyse/prices.csv'),
]
[ds.load() for ds in datasets];

In [ ]:
def get_predictors(n_features, n_timestamps=7, n_classes=3):
    return [
        SimpleLSTM(n_timestamps=n_timestamps, n_features=n_features, n_classes=n_classes),
        MLPClassifier(),
    ]

evaluator = Evaluator('final', datasets, get_predictors, '../reports', seed=42, store=True,
                      n_train_samples=300000, n_test_samples=30000)
metrics = evaluator()
metrics.to_csv("../reports/custom/final.csv")

# plotter.plot_history(evaluator._temp_pipelines['OC-3C']['MLP'][0].steps[-1][1].history, 'MLP on OC-3C')
# plotter.plot_history(evaluator._temp_pipelines['OC-3C']['SLSTM'][0].steps[-1][1].history, 'SLSTM on OC-3C')